# Teste de métodos

In [65]:
import pandas as pd
import numpy as np

In [2]:
@staticmethod
def cep_validator(cep) -> bool:
    """
    Verifica se um CEP está no formato esperado. Retorna False para células vazias.

    :param cep: CEP sem máscara
    :return: Se está no formato esperado
    """
    if pd.isna(cep):
        return False
    if len(cep) == 8 and cep.isdecimal():
        return True
    return False

In [8]:
cep_validator('60355632')

True

In [16]:
@staticmethod
def email_validator(email) -> bool:
    """
    Verifica se um email está no formato esperado. Retorna False para células vazias.

    :param email: CEP sem máscara
    :return: Se está no formato esperado
    """
    if pd.isna(email):
        return False
    if "@" in email and "." in email.split("@")[-1] and ".com" in email:
        return True
    return False

In [17]:
email_validator('carolina.nep@gmi.l')

False

In [21]:
VALID_DDDS = {
    "11",
    "12",
    "13",
    "14",
    "15",
    "16",
    "17",
    "18",
    "19",
    "21",
    "22",
    "24",
    "27",
    "28",
    "31",
    "32",
    "33",
    "34",
    "35",
    "37",
    "38",
    "41",
    "42",
    "43",
    "44",
    "45",
    "46",
    "47",
    "48",
    "49",
    "51",
    "53",
    "54",
    "55",
    "61",
    "62",
    "63",
    "64",
    "65",
    "66",
    "67",
    "68",
    "69",
    "71",
    "73",
    "74",
    "75",
    "77",
    "79",
    "81",
    "82",
    "83",
    "84",
    "85",
    "86",
    "87",
    "88",
    "89",
    "91",
    "92",
    "93",
    "94",
    "95",
    "96",
    "97",
    "98",
    "99",
}

In [22]:
@staticmethod
def ddd_transformer(ddd):
  """
  Transforma um DDD apenas nos seus dois últimos digitos. Retorna-se vazio para DDDs com menos de 2 dígitos,
  com valores inexistentes ou com valores vazios.

  :param ddd: DDD
  :return: DDD formatado
  """
  if pd.isna(ddd):
      return np.nan
  if len(ddd) >= 2:
      clean_ddd = ddd[-2:]
      if clean_ddd in VALID_DDDS:
          return clean_ddd
  return np.nan

In [29]:
ddd_transformer("82085")

'85'

In [30]:
@staticmethod
def cnpj_basic_validator(cnpj_basic) -> bool:
    """
    Checa se um CNPJ básico é válido. Retorna False para células vazias.

    :param cnpj_basic: Valor do CNPJ básico
    :return: Se o CNPJ básico é válido
    """
    if not pd.isna(cnpj_basic) and len(cnpj_basic) == 8 and cnpj_basic.isdecimal():
        return True
    return False

In [36]:
cnpj_basic_validator('12528708')

True

In [51]:
from datetime import datetime

In [52]:
@staticmethod
def date_validator(date, expected_format: str = "%Y%m%d") -> bool:
    """
    Verifica se uma data em string está no formato esperado. Retorna False para células vazias.

    :param date: Data em string
    :param expected_format: Formato esperado
    :return: Se está no formato esperado
    """
    if pd.isna(date) or len(date) != 8:
        return False
    try:
        datetime.strptime(date, expected_format)
        return True
    except ValueError:
        return False

In [58]:
date_validator("20221202")

True

In [71]:
@staticmethod
def get_section_from_division(division: str):
    """
    Recebe a divisão de um CNAE e retorna a seção a qual essa divisão pertence. Limites definidos com base no
    site https://concla.ibge.gov.br/busca-online-cnae.html?view=estrutura. Caso não pertença a nenhuma seção,
    retorna-se uma célula vazia.

    :param str division: Código da divisão
    :return: Letra correspondente à seção ou vazio
    """
    division_num = int(division)
    sections = {
        "A": (1, 3),
        "B": (5, 9),
        "C": (10, 33),
        "D": (35, 35),
        "E": (36, 39),
        "F": (41, 43),
        "G": (45, 47),
        "H": (49, 53),
        "I": (55, 56),
        "J": (58, 63),
        "K": (64, 66),
        "L": (68, 68),
        "M": (69, 75),
        "N": (77, 82),
        "O": (84, 84),
        "P": (85, 85),
        "Q": (86, 88),
        "R": (90, 93),
        "S": (94, 96),
        "T": (97, 97),
        "U": (99, 99),
    }

    for section, division_limits in sections.items():
        if division_num in range(division_limits[0], division_limits[1] + 1):
            return section

    return np.nan

In [76]:
def cnae_categories_cretator(cnae: str):
    """
    Define a seção, divisão, grupo e classe de um CNAE com base no seu valor.
    Caso o CNAE seja "8888888" (Atividade econômica não informada) atribui-se vazio para essas categorias.

    :param str cnae: Código do CNAE
    :return: Lista de strings contendo os códigos de seção, divisão, grupo e classe
    """
    if pd.isna(cnae) or cnae == "8888888":
        return [np.nan, np.nan, np.nan, np.nan]

    divisao, grupo, classe = cnae[:2], cnae[:3], cnae[:5]
    secao = get_section_from_division(divisao)
    return [secao, divisao, grupo, classe]

In [77]:
cnae_categories_cretator("7020400")

['M', '70', '702', '70204']

In [78]:
@staticmethod
def money_validator(value) -> bool:
    """
    Checa se um valor é uma quantia monetária válida. Assume-se o padrão brasileiro de duas casas decimais
    para indicar a quantia de centavos e vírgula como separador. Retorna False para células vazias.

    :param value: Valor monetário em string
    :return: Se o valor é válido
    """
    if pd.isna(value):
        return False
    if "," not in value:
        return False
    real, centavos = value.split(",")
    if not (real.isdecimal() and centavos.isdecimal()):
        return False
    if len(centavos) != 2:
        return False
    return True

In [81]:
money_validator('100,00')

True